In [6]:
import pymongo

db = 'python_dev'

In [7]:
#connect to database
myclient = pymongo.MongoClient("mongodb://localhost:27017")

print(myclient)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


In [8]:
mydb = myclient[db]

In [9]:
# return a list of databases
print(myclient.list_database_names())

['Catalog', 'ProductDB', 'admin', 'config', 'local', 'python_dev']


In [10]:
# create a collection called "customers"

mycol = mydb["customers"]

In [11]:
# Insert a new record in customer's collection

mydict = { "name" : "john", "age" : 36}

x = mycol.insert_one(mydict)

print(x.inserted_id)

63909adb5384327ed7b675a4


In [31]:
# check mongo repository pattern

from abc import abstractmethod, abstractproperty
from typing import Generic, TypeVar
from collections import namedtuple


class User:
    id: str
    name: str

class Customer:
    id: str
    name: str

T = TypeVar('T',bound=Customer)
C = TypeVar('C')

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


class MongoRepository(Generic[T]):

    def __init__(self,collection_name) -> None:
        self.client = pymongo.MongoClient('mongodb://localhost:27017')
        self.database = self.client[db]
        self.collection = self.database[collection_name]
        super().__init__()

    @abstractmethod
    def find(self) -> list[T]:
        return list(map(lambda x: dotdict(x), self.collection.find()))

    @abstractmethod
    def find_one(self) -> T:
        return dotdict(self.collection.find_one())


class CustomerRepository(MongoRepository[Customer]):
        def __init__(self) -> None:
             super().__init__('customers')


customers = CustomerRepository()


findAll = customers.find()
findOne = customers.find_one()


print(findAll)
print(findOne)

<slot wrapper '__str__' of 'object' objects>
True
[{'_id': ObjectId('639082c1d4a29724d066eb71'), 'name': 'john', 'age': 36}, {'_id': ObjectId('6390869ed4a29724d066eb75'), 'name': 'john', 'age': 36}, {'_id': ObjectId('63909adb5384327ed7b675a4'), 'name': 'john', 'age': 36}]
{'_id': ObjectId('639082c1d4a29724d066eb71'), 'name': 'john', 'age': 36}
